In [1]:
%pip install --upgrade openai
%pip install claudette
%pip install python-dotenv
%pip install -U bitsandbytes
%pip install optimum
%pip install auto-gptq

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tokenize, ast
from io import BytesIO
import os

In [3]:
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch
import torch

In [4]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [17]:
mn = "meta-llama/Llama-2-7b-hf"

In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [6]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,quantization_config=quantization_config)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [18]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Silver Rubanza is a "
toks = tokr(prompt, return_tensors="pt")

In [8]:
toks

{'input_ids': tensor([[    1, 16466, 14985,  8520,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [9]:
tokr.batch_decode(toks['input_ids'])

['<s> Silver Rubanza is a ']

In [10]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

CPU times: user 1.9 s, sys: 393 ms, total: 2.29 s
Wall time: 2.36 s


tensor([[    1, 16466, 14985,  8520,   338,   263, 29871, 29906, 29900, 29896,
         29955,  6345, 19287,   491, 16466,   383,   392,  4524,   714,   310,
          1749,  1914]])

In [11]:
tokr.batch_decode(res)

['<s> Silver Rubanza is a 2017 born mare by Silver Fandango out of our own']

In [12]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', '

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [14]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1581: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


CPU times: user 365 ms, sys: 0 ns, total: 365 ms
Wall time: 363 ms


tensor([[    1, 16466, 14985,  8520,   338,   263, 29871, 29896, 29900, 29900,
         29995, 13283, 19480,  7371,   491,   278,  7021, 11443,   341,   524,
         29889,   450]])

In [15]:
tokr.batch_decode(res)

['<s> Silver Rubanza is a 100% silver coin produced by the Royal Canadian Mint. The']

In [7]:
def gen(p, maxlen=15, sample=True):
  toks = tokr(p,return_tensors="pt")
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
  return tokr.batch_decode(res)

In [17]:
%%time
gen(prompt,50)

['<s> Silver Rubanza is a 25-year-old rapper from Harlem, New York.\n\nDoja Cat is a 25-year-old rapper from Los Angeles, California\n\n2 Chainz is a 40-year-']

# Other models
Stable Beluga


In [7]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
sb_sys = "### System:\n You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. . \n\n"

In [10]:
def mk_prompt(user, syst=sb_sys):
    return f"{syst} ### User: {user}\n\n### Assistant:\n"

In [11]:

#sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [11]:
#def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [15]:
ques = "Who is Silver Rubanza?"

In [15]:
%%time
gen(mk_prompt(ques), 150)

CPU times: user 2.92 s, sys: 280 ms, total: 3.2 s
Wall time: 3.27 s


['<s> ### System:\n You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. . \n\n ### User: Who is Silver Rubanza?\n\n### Assistant:\n Silver Rubanza is a fictional character created by the German author Paul Ernst. He is the main protagonist in the novel "The Saga of Silver Rubanza", published in 1935.</s>']

Open Orca /Playtpus 2

In [12]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/OpenOrca-Platypus2-13B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.b

In [13]:
def mk_oo_prompt(user):
    return f"### Instruction: {user}\n\n ### Response:\n"

In [19]:
gen(mk_oo_prompt(ques),150)

["<s> ### Instruction: Who is Silver Rubanza?\n\n ### Response:\n\n Silver Rubanza is a Belgian fashion designer, known for creating unique and exclusive luxury garments for high-profile clients, including celebrities and royalty. He has designed various fashion items, from haute couture to accessories like bags and hats. Rubanza's designs have been showcased in prestigious fashion events and exhibitions.\n\nHis innovative aesthetic style often draws inspiration from distinct elements, such as nature, art, and culture, integrating them into his designs. Silver Rubanza has established his name in the fashion world with his skillful craftsmanship and exceptional creations.\n\nPlease note that there might be minimal variations in the"]

# Retieval Augmented Generation

In [21]:
%pip install Wikipedia-API

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15446 sha256=459fa355244c88527bf0e756c833853e2ca3ae4c15395c9747c7d076efe24092
  Stored in directory: /home/zeus/.cache/pip/wheels/1d/f8/07/0508c38722dcd82ee355e9d85e33c9e9471d4bec0f8ae72de0
Successfully built Wikipedia-API
Note: you may need to restart the kernel to use updated packages.


In [22]:
from wikipediaapi import Wikipedia

In [23]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [29]:
jh_page

'Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator. \nHe is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.\nPreviously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.\nEarly in the COVID-19 epidemic he was a leading advocate for masking.\n\nEarly life\nHoward was born in London, United Kingdom, and moved to Melbourne, Australia in 1976. He attended Melbourne Grammar and studied philosophy at the University of Melbourne.\n\nCareer\nHoward started his career in management consulting, working at McKinsey & Co and AT Kearney. He remained in management consulting for eight years before becoming an entrepreneur.\nEarly in his career, Howard contributed to open-source projects, particularly the Perl programming language, Cyrus IMAP server, and Postfix SMTP server. He helped develop the Perl 

In [24]:
print(jh_page[:500])

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator. 
He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.

Early life
Howard was born in London, United Kingdom, and mo


In [30]:
print(type(jh_page))

<class 'str'>


In [31]:
sr_page = 'Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work'

In [32]:
print(type(sr_page))

<class 'str'>


In [33]:
len(jh_page.split()),len(sr_page.split())

(616, 98)

In [34]:
ques_ctx = f"""Answer the question with the help of the provided context.

## Context
{sr_page}

## Question
{ques}

"""

In [35]:
res = gen(mk_prompt(ques_ctx),300)

In [36]:
print(res[0].split('### Assistant:\n')[1])

Silver Rubanza is an individual with a background in Software Engineering, having experience in various steps of the machine learning lifecycle, including Exploratory Data Analysis, Data wrangling, Feature Engineering, Model building, deployment, testing, and monitoring. Additionally, Rubanza Silver is proficient in Python, PyTorch, and different libraries such as fastai and sklearn. Furthermore, Rubanza Silver has over 6 years of experience creating software applications using HTML, CSS, Javascript, React JS, and other languages. 

Reference: https://ai497.github.io/help-assistant/api/assistant/user/3/</s>


In [ ]:
%pip install sentence-transformers

In [39]:
from sentence_transformers import SentenceTransformer

In [40]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
jh = jh_page.split('\n\n')[0]
print(jh)

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator. 
He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.


In [42]:
sr_page

'Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work'

In [44]:
sr = sr_page.split('\n\n')[0]
print(sr)

Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work


In [45]:
sr

'Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work'

In [46]:
q_emb,jh_emb,sr_emb = emb_model.encode([ques,jh,sr], convert_to_tensor=True)

In [47]:
sr_emb.shape,jh_emb.shape

(torch.Size([384]), torch.Size([384]))

In [48]:
import torch.nn.functional as F

In [49]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

tensor(0.4381, device='cuda:0')

In [50]:
F.cosine_similarity(q_emb, sr_emb, dim=0)

tensor(0.6374, device='cuda:0')

Finetuning

In [51]:
import datasets

In [52]:
ds = datasets.load_dataset('knowrohit07/know_medical_dialogue_v2')

README.md:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

know_med_v4.json:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6307 [00:00<?, ? examples/s]

In [53]:
ds

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 6307
    })
})

In [55]:
trn = ds['train']
trn[3]

{'output': 'again, really need complete work up and medical notes to provide any insight here',
 'input': '',
 'instruction': "Appreciate it al labs have come back completely negative only thing that's been consistently elevated but wasn't that much of a concern was CK levels everything else you could think of from any type of test normal"}

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])